In [ ]:
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
%matplotlib inline
import numpy as np
import os
import pandas as pd
import plotly.offline as py
import plotly.graph_objs as go
import seaborn as sns

plt.rcParams['figure.figsize']=(20,10)
print(os.listdir("../input"))
py.init_notebook_mode(connected=False)

%env JOBLIB_TEMP_FOLDER=/tmp

## Step 1: Import the Data

In [ ]:
df = pd.read_csv('../input/labels.csv')

In [ ]:
df.head()

In [ ]:
df['breed'].describe()

## Step 2: Explore the Data
### Let's visualize the data, get an intuition of the distribution

In [ ]:
temp = pd.DataFrame({'breed': df['breed'].value_counts().index, 'instances': df['breed'].value_counts().values})
temp = temp.sort_values(by=['breed'])
temp.head()

In [ ]:
trace = go.Bar(x=temp['breed'], y=temp['instances'])
data = [trace]
layout = go.Layout(
        title='Breed Counts',
        autosize=False,
        width=5000,
        height=500,
        margin=dict(
            l=100,
            r=100,
            b=100,
            t=100
        )
    )
fig = go.Figure(data=data, layout=layout)
py.iplot(fig)

As we can see, almost all the breeds have sufficient enough training images. Hence, we won't have to generate additional data right now.

### Step 3: One-Hot Encode the Breed columns and prepare the data for a classifier

In [ ]:
df['breed'] = pd.Categorical(df['breed'])
df['breed'] = df['breed'].cat.codes

In [ ]:
df.head()

### Let's create the train data by creating a torch dataset

In [ ]:
import torch
import torchvision
from torchvision import transforms
from torchvision.datasets import ImageFolder
from torch.utils.data import DataLoader
from torch.utils.data.sampler import SubsetRandomSampler
from torch.utils.data.dataset import Dataset
import torch.nn.functional as F
from PIL import Image

import torchvision.models as models
import torch.nn as nn
import torch.optim as optim

In [ ]:
df['image_path'] = '../input/train/' + df['id'].astype(str) + '.jpg'

In [ ]:
df.head()

In [ ]:
labels = torch.tensor(df['breed'].tolist())

In [ ]:
class CustomDataset(Dataset):
    def __init__(self, image_path, labels=[], train=True):
        self.image_path = image_path
        self.labels = labels
        self.transform = transforms.Compose([
                        transforms.Resize(255),
                        transforms.CenterCrop(224),
                        transforms.ToTensor(),
                        transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
                    ])

    def __getitem__(self, index):
        image = Image.open(self.image_path[index])
        t_image = self.transform(image)
        
        if len(self.labels) > 0:
            return t_image, self.labels[index]
        return t_image

    def __len__(self):  # return count of sample we have
        return len(self.image_path)

In [ ]:
dataset = CustomDataset(df['image_path'], labels, train=True)
# train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True, num_workers=1)
batch_size = 32
validation_split = .2
shuffle_dataset = True
random_seed= 42

dataset_size = len(dataset)
indices = list(range(dataset_size))
split = int(np.floor(validation_split * dataset_size))

if shuffle_dataset :
    np.random.seed(random_seed)
    np.random.shuffle(indices)
train_indices, val_indices = indices[split:], indices[:split]

In [ ]:
train_sampler = SubsetRandomSampler(train_indices)
valid_sampler = SubsetRandomSampler(val_indices)

train_loader = DataLoader(dataset, batch_size=batch_size, 
                                           sampler=train_sampler)
validation_loader = DataLoader(dataset, batch_size=batch_size,
                                                sampler=valid_sampler)


In [ ]:
class Classifier(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 = nn.Conv2d(3, 16, kernel_size=3, stride=1, padding=0)
        self.pool1 = nn.MaxPool2d(kernel_size=2, stride=1, padding=0)
        self.conv2 = nn.Conv2d(16, 64, kernel_size=5, stride=1, padding=0)
        self.pool2 = nn.MaxPool2d(kernel_size=2, stride=1, padding=0)
        self.fc1 = nn.Linear(216*216*64, 1024)
        self.fc2 = nn.Linear(1024, 512)
        self.fc3 = nn.Linear(512, 120)
        
    def forward(self, x):
        x = F.relu(self.conv1(x))
        x = F.relu(self.conv2(x))
        x = x.view(-1, 64*216*216)
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)

        return x

In [ ]:
def outputSize(in_size, kernel_size, stride, padding):
    output = int((in_size - kernel_size + 2*(padding)) / stride) + 1
    return(output)

In [ ]:
ot1 = outputSize(224, 3, 1, 0)
ot2 = outputSize(ot1, 2, 1, 0)
ot3 = outputSize(ot2, 5, 1, 0)
ot4 = outputSize(ot3, 2, 1, 0)
ot4

In [ ]:
model = Classifier()
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
# Move model to the device specified above
model.to(device)


In [ ]:
criterion = nn.NLLLoss()
# Set the optimizer function using torch.optim as optim library
optimizer = optim.Adam(model.parameters())

In [ ]:
epochs = 10
for epoch in range(epochs):
    train_loss = 0
    val_loss = 0
    accuracy = 0
    
    # Training the model
    model.train()
    counter = 0
    for inputs, labels in train_loader:
        # Move to device
        inputs, labels = inputs.to(device), labels.to(device)
        if epoch == 0:
                print('IMG1: ', inputs[0].shape)
        # Clear optimizers
        optimizer.zero_grad()
        # Forward pass
        output = model.forward(inputs)
        # Loss
        loss = criterion(output, labels)
        # Calculate gradients (backpropogation)
        loss.backward()
        # Adjust parameters based on gradients
        optimizer.step()
        # Add the loss to the training set's rnning loss
        train_loss += loss.item()*inputs.size(0)
        
        # Print the progress of our training
        counter += 1
        #print(counter, "/", len(train_loader))
        
        # Evaluating the model
    model.eval()
    counter = 0
    # Tell torch not to calculate gradients
    with torch.no_grad():
        for inputs, labels in validation_loader:
            # Move to device
            inputs, labels = inputs.to(device), labels.to(device)
            # Forward pass
            output = model.forward(inputs)
            # Calculate Loss
            valloss = criterion(F.log_softmax(output), labels)
            # Add loss to the validation set's running loss
            val_loss += valloss.item()*inputs.size(0)
            
            # Since our model outputs a LogSoftmax, find the real 
            # percentages by reversing the log function
            output = torch.exp(output)
            # Get the top class of the output
            top_p, top_class = output.topk(1, dim=1)
            # See how many of the classes were correct?
            equals = top_class == labels.view(*top_class.shape)
            # Calculate the mean (get the accuracy for this batch)
            # and add it to the running accuracy for this epoch
            accuracy += torch.mean(equals.type(torch.FloatTensor)).item()
            
            # Print the progress of our evaluation
            counter += 1
            #print(counter, "/", len(val_loader))
    
    # Get the average loss for the entire epoch
    train_loss = train_loss/len(train_loader.dataset)
    valid_loss = val_loss/len(validation_loader.dataset)
    # Print out the information
    print('Accuracy: ', accuracy/len(validation_loader))
    print('Epoch: {} \tTraining Loss: {:.6f} \tValidation Loss: {:.6f}'.format(epoch, train_loss, valid_loss))


In [ ]:
# # Saving the model
# checkpoint = {'model': model,
#               'state_dict': model.state_dict(),
#               'optimizer' : optimizer.state_dict()}

# torch.save(checkpoint, 'model1_checkpoint.pth')

In [ ]:
def load_checkpoint(filepath):
    checkpoint = torch.load(filepath)
    model = checkpoint['model']
    model.load_state_dict(checkpoint['state_dict'])
    for parameter in model.parameters():
        parameter.requires_grad = False
    
    model.eval()
    
    return model

In [ ]:
model = load_checkpoint('model1_checkpoint.pth')
print(model)

In [ ]:
df_submission = pd.read_csv('../input/sample_submission.csv')

In [ ]:
df_submission.head()

In [ ]:
test_image_paths = '../input/test/' + df_submission['id'].astype(str) + '.jpg'

In [ ]:
test_dataset = CustomDataset(test_image_paths, train=False)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)

In [ ]:
pred_labels = []
with torch.no_grad():
    model.cuda()
    model.eval()
    for images in test_loader:
        images.cuda()
        output = model.forward(images)
        output = torch.exp(output)
        print('OUTPUT :', output)
        pred_labels.append(output)

In [ ]:
len(pred_labels)